In [1]:
!pip install safetensors

In [12]:
import torch
from torch import nn
from transformers import BartForConditionalGeneration, BartConfig
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding
from typing import List, Optional, Tuple
from transformers import PreTrainedTokenizerFast
from safetensors.torch import load_file

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 모델 구조 로드

In [67]:
# Longformer Attention 레이어 정의
class LongformerAttention(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.embed_dim = config.d_model
        self.longformer_self_attn = LongformerSelfAttention(config, layer_id=layer_id)
        self.output_projection = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:

        if attention_mask is not None:
            attention_mask = attention_mask.squeeze(dim=1)
            attention_mask = attention_mask[:, 0]

        is_index_masked = attention_mask < 0 if attention_mask is not None else None
        is_index_global_attn = attention_mask > 0 if attention_mask is not None else None
        is_global_attn = is_index_global_attn.flatten().any().item() if attention_mask is not None else False

        outputs = self.longformer_self_attn(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            is_index_masked=is_index_masked,
            is_index_global_attn=is_index_global_attn,
            is_global_attn=is_global_attn,
            output_attentions=output_attentions,
        )

        attention_output = self.output_projection(outputs[0])
        return (attention_output,) + outputs[1:] if len(outputs) == 2 else (attention_output, None, None)

In [68]:
# Longformer Attention 적용 KoBART 모델 정의
class KoBARTWithLongformer(BartForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)

        if config.attention_mode == 'n2':
            pass
        else:
            # 포지셔널 임베딩 확장 (encoder)
            self.model.encoder.embed_positions = BartLearnedPositionalEmbedding(
                config.max_encoder_position_embeddings,  # num_embeddings
                config.d_model  # embedding_dim
            )

            # 인코더에 Longformer Attention 적용
            for i, encoder_layer in enumerate(self.model.encoder.layers):
                encoder_layer.self_attn = LongformerAttention(config, layer_id=i)

In [69]:
# Longformer Attention 적용 KoBART 설정
class KoBARTConfig(BartConfig):
    def __init__(self, attention_window: List[int] = None,
                 attention_dilation: List[int] = None,
                 autoregressive: bool = False,
                 attention_mode: str = 'sliding_chunks',
                 gradient_checkpointing: bool = False,
                 attention_probs_dropout_prob: float = 0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self.attention_window = attention_window
        self.attention_dilation = attention_dilation
        self.autoregressive = autoregressive
        self.attention_mode = attention_mode
        self.gradient_checkpointing = gradient_checkpointing
        self.attention_probs_dropout_prob = attention_probs_dropout_prob

        assert self.attention_mode in ['sliding_chunks', 'n2']

In [70]:
# 모델 초기화 함수
def initialize_kobart_with_longformer():
    attention_window_size = 512  # 짝수
    max_position_embeddings = 4104  # 1026의 배수

    # KoBART 기본 설정
    config = KoBARTConfig(
        vocab_size=30000,  # KoBART 기본 vocab 크기
        d_model=768,  # Hidden Dimension
        encoder_layers=6,  # 인코더 레이어 수
        decoder_layers=6,  # 디코더 레이어 수
        encoder_attention_heads=16,  # Attention Head 수
        decoder_attention_heads=16,  # Attention Head 수
        encoder_ffn_dim=3072,  # FFN 차원
        decoder_ffn_dim=3072,  # FFN 차원
        attention_window=[attention_window_size] * 6,  # Longformer Attention Window 설정
        max_encoder_position_embeddings=max_position_embeddings,  # 확장된 포지셔널 임베딩
        max_decoder_position_embeddings=1026,  # 디코더 크기 유지
        attention_mode='sliding_chunks',
        gradient_checkpointing=True,
        attention_probs_dropout_prob=0.1  # 드롭아웃 설정
    )

    # Tokenizer 및 모델 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
    model = KoBARTWithLongformer.from_pretrained(
        "gogamza/kobart-base-v2",
        config=config,
        ignore_mismatched_sizes=True  # 초기 로드 시 불일치 무시
    )

    # 인코더 포지셔널 임베딩 확장
    current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
    print(f"Current encoder max pos: {current_max_pos}")
    new_encoder_pos_embed = model.model.encoder.embed_positions.weight.new_empty(
        config.max_encoder_position_embeddings + 2, embed_size
    )
    # 기존 가중치 복사
    new_encoder_pos_embed[:current_max_pos] = model.model.encoder.embed_positions.weight
    # 확장된 부분 랜덤 초기화
    nn.init.normal_(new_encoder_pos_embed[current_max_pos:], mean=0.0, std=0.02)
    model.model.encoder.embed_positions.weight.data = new_encoder_pos_embed

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    return model, tokenizer

#### 저장된 가중치 로드

In [71]:
# 모델 가중치 로드 함수
def load_model_weights(model, model_path):
    # 모델에 가중치 로드
    state_dict = load_file(f"{model_path}/model.safetensors")
    model.load_state_dict(state_dict, strict=False)

    return model

# 저장된 모델 경로
model_path = "/content/drive/MyDrive/응용자연어처리/project/Longformer_KoBART_finetuning_v6"

# 저장된 가중치와 토크나이저 로드
model, tokenizer = initialize_kobart_with_longformer()

model = load_model_weights(model, model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of KoBARTWithLongformer were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['encoder.layers.0.self_attn.longformer_self_attn.key.bias', 'encoder.layers.0.self_attn.longformer_self_attn.key.weight', 'encoder.layers.0.self_attn.longformer_self_attn.key_global.bias', 'encoder.layers.0.self_attn.longformer_self_attn.key_global.weight', 'encoder.layers.0.self_attn.longformer_self_attn.query.bias', 'encoder.layers.0.self_attn.longformer_self_attn.query.weight', 'encoder.layers.0.self_attn.longformer_self_attn.query_global.bias', 'encoder.layers.0.self_attn.longformer_self_attn.query_global.weight', 'encoder.layers.0.self_attn.longformer_self_attn.value.bias', 'encoder.layers.0.self_attn.longformer_self_attn.value.weight', 'encoder.layers.0.self_attn.longformer_self_attn.value_globa

Current encoder max pos: 4106


#### 요약문 생성

In [47]:
'''
# 요약문 생성 함수
def generate_summary(model, tokenizer, text):
    device = next(model.parameters()).device
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding="max_length"
    ).to(device)
    inputs.setdefault("attention_mask", (inputs["input_ids"] != tokenizer.pad_token_id).long())

    summary_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
'''

In [51]:
from torch.utils.data import DataLoader
import pandas as pd

In [52]:
# 테스트 데이터셋 로드
df = pd.read_csv("/content/drive/MyDrive/응용자연어처리/project/data/long_test.csv")

In [54]:
# 배치 요약문 생성 함수
def generate_batch_summaries(model, tokenizer, texts, device, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size)
    summaries = []
    for batch in dataloader:
        # 텍스트를 배치로 토큰화
        inputs = tokenizer(
            list(batch),
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding="max_length"
        ).to(device)

        # 모델로 요약 생성
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        # 요약문 디코딩
        batch_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summaries.extend(batch_summaries)
    return summaries

# GPU/CPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 배치 처리 및 요약문 생성
batch_size = 16
df["generated_summary"] = generate_batch_summaries(
    model, tokenizer, df["contents"].tolist(), device, batch_size
)

In [57]:
df.head()

,contents,summary,char_length,length,generated_summary
0,"122개 公기관, 지방 추가이전 시급 혁신도시 활성화 대책도 마련해야 지역균형발전은...",지역균형발전은 분권과 함께 노무현정부의 화두였지만 결론적으로는 수도권 집중이 심화되...,1765,long,문재인 대통령은 취임때 분권형 개헌까지 거론할 정도로 분권과 지역균형발전에 관심을 ...
1,5일 오전 9시쯤 대구 동산병원 장례식장 백합관 지하 1층 분향실 복도. 지나다니는...,독도 소방헬기 추락사고 희생자인 중앙119구조본부 소방공무원과 소방항공대원들의 빈소...,1520,long,5일 오전 9시쯤 대구 동산병원 장례식장 백합 지하 1층 분향실에서 아직 유족들의 ...
2,정영식 사진=최종훈 인스타그램 캡처 음주운전과 사건무마 청탁 의혹이 불거진 밴드 F...,"14일 소속사 FNC엔터테인먼트는 ""최근 잇따른 사건에 연루돼 물의를 빚은 최종훈에...",1740,long,FT아일랜드의 최종훈(29)이 연예계 은퇴 의사를 밝혔지만 소속사 FNC엔터테인먼트...
3,"李시장 “4차 산업혁명, 日 뛰어넘는 기회…‘AI 추진단’ 발족” 金지사 “아베정부...",김영록 전남지사가 간부회의에서 4차 산업혁명은 일본을 뛰어넘을 수 있는 절호의 기회...,1568,long,"김영록 전남지사가 6일 도청 서재필실에서 일본 수출규제와 관련해 민관협의회를 갖고,..."
4,공정거래위원회가 내년 1분기에 정보통신기술(ICT) 전담팀 내에 반도체분과를 신설한...,조성욱 공정거래위원장은 19일 세종정부청사에서 열린 취임 100일 기자간담회에서 4...,1728,long,공정거래위원회가 내년 1분기에 정보통신기술(ICT) 전담팀 내에 반도체 분과를 신설...


In [58]:
summaries = df["summary"]
generated_summaries = df["generated_summary"]

#### 성능평가 - long

ROUGE: https://github.com/HeegyuKim/korouge

In [33]:
!pip install korouge_score

In [59]:
import numpy as np
from korouge_score import rouge_scorer

In [60]:
# 한국어 ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])

In [61]:
# ROUGE 점수 계산
rouge_scores = []
for reference, generated in zip(df["summary"], df["generated_summary"]):
    scores = scorer.score(reference, generated)
    rouge_scores.append(scores)

# 결과 확인
for i, scores in enumerate(rouge_scores[:5]):
    print(f"Sample {i + 1}:")
    print(scores)
    print()

Sample 1:
{'rouge1': Score(precision=0.2727272727272727, recall=0.3157894736842105, fmeasure=0.2926829268292683), 'rouge2': Score(precision=0.047619047619047616, recall=0.05555555555555555, fmeasure=0.05128205128205128), 'rougeL': Score(precision=0.2727272727272727, recall=0.3157894736842105, fmeasure=0.2926829268292683), 'rougeLsum': Score(precision=0.2727272727272727, recall=0.3333333333333333, fmeasure=0.3)}

Sample 2:
{'rouge1': Score(precision=0.02631578947368421, recall=0.029411764705882353, fmeasure=0.027777777777777776), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.02631578947368421, recall=0.029411764705882353, fmeasure=0.027777777777777776), 'rougeLsum': Score(precision=0.02631578947368421, recall=0.029411764705882353, fmeasure=0.027777777777777776)}

Sample 3:
{'rouge1': Score(precision=0.75, recall=0.5714285714285714, fmeasure=0.6486486486486486), 'rouge2': Score(precision=0.6451612903225806, recall=0.4878048780487805, fmeasure=0.555

In [77]:
# ROUGE 점수 평균 계산 (fmeasure 평균)
metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
average_scores = {metric: 0 for metric in metrics}

# 총 샘플 수
n = len(rouge_scores)

# 각 메트릭의 fmeasure 합산
for score in rouge_scores:
    for metric in metrics:
        average_scores[metric] += getattr(score[metric], "fmeasure", 0)

# 각 메트릭의 평균 계산
for metric in metrics:
    average_scores[metric] /= n

print(average_scores)

{'rouge1': 0.22405525580358793, 'rouge2': 0.11461126293569565, 'rougeL': 0.2012439795341342, 'rougeLsum': 0.20133272141849934}


BERT-Score: https://github.com/Tiiiger/bert_score

In [78]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [83]:
# BERT-Score
bert_tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
bert_model = BertModel.from_pretrained("beomi/kcbert-base").eval()

def get_bert_embeddings(sentence, tokenizer, model):
    # 문장을 토큰화하고 BERT 입력 형식으로 변환
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

    # BERT 모델을 통해 문장의 임베딩 얻기
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)

    return embeddings.numpy()

def calculate_bert_score(sentence1, sentence2, tokenizer, model):
    # 각 문장의 BERT 임베딩 벡터 얻기
    embedding1 = get_bert_embeddings(sentence1, tokenizer, model)
    embedding2 = get_bert_embeddings(sentence2, tokenizer, model)

    # 코사인 유사도 계산 (1 - cosine distance)
    score = 1 - cosine(embedding1, embedding2)

    return score

In [84]:
# BERT-Score 점수 계산
df["bert_score"] = df.apply(lambda row: calculate_bert_score(row["summary"], row["generated_summary"], bert_tokenizer, bert_model), axis=1)
print(df.head())

                                            contents  \
0  122개 公기관, 지방 추가이전 시급 혁신도시 활성화 대책도 마련해야 지역균형발전은...   
1  5일 오전 9시쯤 대구 동산병원 장례식장 백합관 지하 1층 분향실 복도. 지나다니는...   
2  정영식 사진=최종훈 인스타그램 캡처 음주운전과 사건무마 청탁 의혹이 불거진 밴드 F...   
3  李시장 “4차 산업혁명, 日 뛰어넘는 기회…‘AI 추진단’ 발족” 金지사 “아베정부...   
4  공정거래위원회가 내년 1분기에 정보통신기술(ICT) 전담팀 내에 반도체분과를 신설한...   

                                             summary  char_length length  \
0  지역균형발전은 분권과 함께 노무현정부의 화두였지만 결론적으로는 수도권 집중이 심화되...         1765   long   
1  독도 소방헬기 추락사고 희생자인 중앙119구조본부 소방공무원과 소방항공대원들의 빈소...         1520   long   
2  14일 소속사 FNC엔터테인먼트는 "최근 잇따른 사건에 연루돼 물의를 빚은 최종훈에...         1740   long   
3  김영록 전남지사가 간부회의에서 4차 산업혁명은 일본을 뛰어넘을 수 있는 절호의 기회...         1568   long   
4  조성욱 공정거래위원장은 19일 세종정부청사에서 열린 취임 100일 기자간담회에서 4...         1728   long   

                                   generated_summary  bert_score  
0  문재인 대통령은 취임때 분권형 개헌까지 거론할 정도로 분권과 지역균형발전에 관심을 ...    0.913023  
1  5일 오전 9시쯤 대구 동산병원 장례식장 백합 지하 1층 분향실에서 아직 유족들의 ...    0.808013  
2  FT

In [85]:
# BERT-Score 평균
df["bert_score"].mean()

0.8405518694743583